<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/figures/ch19_multivariableIntegration_figures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 19 (Multivariable integration)

---

# About this code file:

### This notebook will reproduce the figures in this chapter, and illustrate the mathematical concepts explained in the book. The point of providing the code is not just for you to recreate the figures, but for you to modify, adapt, explore, and experiment with the code.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import scipy.integrate as spi
import matplotlib.pyplot as plt
from IPython.display import Math

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                    #  'axes.spines.right':False,  # remove axis bounding box
                    #  'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Figure 19.1: Definite integral as volume under a surface

In [ ]:
# the function
def fun2xy(x,y): return 10 - x**2/8 - y**2/8

# grid for x and y
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x,y)
Z = fun2xy(X,Y)

# show the plot!
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122,projection='3d')

# plot a regular line
ax1.plot(x,fun2xy(x,0),'k')
ax1.fill_between(np.linspace(-1,3,101),fun2xy(np.linspace(-1,3,101),0),color='k',alpha=.4)
ax1.grid(color=[.7,.7,.7],linestyle='--')
ax1.set(xlim=x[[0,-1]],xlabel='$x$',ylabel='$y$',title=r'$\bf{A}$)  Definite integral as area under a curve')

# the surface and "walls" indicating volume
ax2.plot_surface(X,Y,Z, cmap='gray')
ax2.fill_between(x,y[0],0, x,y[0],fun2xy(x,y[0]), color='k',alpha=.3,linewidth=.5)
ax2.fill_between(x[-1],y,0, x[-1],y,fun2xy(x[-1],y), color='k',alpha=.3,linewidth=.5)

ax2.set(xlabel='$x$',ylabel='$y$',zlabel='$z$',zlim=[0,10],title=r'$\bf{B}$)  Definite integral as volume under a surface')

plt.tight_layout()
plt.savefig('multiint_defIntAreaVolume.png')
plt.show()

# Figure 19.3: sympy output

In [ ]:
x,y = sym.symbols('x,y')

f_xy = 3*x + x*y
F_x = sym.integrate(f_xy,x)
F_y = sym.integrate(f_xy,y)
F_xy = sym.integrate(F_x,y)
F_yx = sym.integrate(F_y,x)

display(Math('f(x,y) = %s' %sym.latex(f_xy))), print('')
display(Math('F_{x} = %s' %sym.latex(F_x))), print('')
display(Math('F_{y} = %s' %sym.latex(F_y))), print('')
display(Math('F_{xy} = %s' %sym.latex(F_xy))), print('')
display(Math('F_{yx} = %s' %sym.latex(F_yx)))

In [ ]:
# lambdify the expressions
f_xy_l = sym.lambdify((x,y),f_xy)
F_x_l  = sym.lambdify((x,y),F_x)
F_y_l  = sym.lambdify((x,y),F_y)
F_xy_l = sym.lambdify((x,y),F_xy)

# Figure 19.4: Indefinite integrals

In [ ]:
# and now visualize
# create coordinate grid matrices
a,b = -10,3
xx = np.linspace(a,b,151)
X,Y = np.meshgrid(xx,xx)

# show the function and its partial integrals
_,axs = plt.subplots(2,2,figsize=(10,6))
axs[0,0].imshow(f_xy_l(X,Y),extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[0,0].set_title(r'$\bf{A}$)  $f(x,y) = %s$' %sym.latex(f_xy))

axs[0,1].imshow(F_x_l(X,Y),extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[0,1].set_title(r'$\bf{B}$)  $F_x = %s$' %sym.latex(F_x))

axs[1,0].imshow(F_y_l(X,Y),extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[1,0].set_title(r'$\bf{C}$)  $F_y = %s$' %sym.latex(F_y))

axs[1,1].imshow(F_xy_l(X,Y),extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[1,1].set_title(r'$\bf{D}$)  $F_{xy} = %s$' %sym.latex(F_xy))

for axi in axs.flatten(): axi.set(xlabel='$x$',ylabel='$y$'), axi.axis('auto')

plt.tight_layout()
plt.savefig('multiint_visualizationEx1sympy.png')
plt.show()

# Figures 19.5-6: Reproducing sympy result with scipy

In [ ]:
# confirming the sympy result
xx = np.linspace(a,b,81)
dx = xx[1] - xx[0]
yy = xx + 0
dy = yy[1] - yy[0]
X,Y = np.meshgrid(xx,yy)
f_xy = f_xy_l(X,Y)

# partial integrals
F_x  = spi.cumulative_simpson(f_xy,dx=dx,axis=1)
F_y  = spi.cumulative_simpson(f_xy,dx=dy,axis=0)
F_xy = spi.cumulative_simpson(F_x,dx=dx,axis=0)

# normalizations (comment these lines for figure 19.5)
F_x  -= F_x[:,np.argmin(abs(xx))].reshape(-1,1)
F_y  -= F_y[np.argmin(abs(xx)),:].reshape(1,-1)
F_xy -= F_xy[np.argmin(abs(xx)),:].reshape(1,-1)
F_xy -= F_xy[:,np.argmin(abs(xx))].reshape(-1,1)

In [ ]:
# and now visualize
_,axs = plt.subplots(2,2,figsize=(10,6))
axs[0,0].imshow(f_xy,extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[0,0].set_title(r'$\bf{A}$)  $f(x,y)$')

axs[0,1].imshow(F_x,extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[0,1].set_title(r'$\bf{B}$)  $F_x $')

axs[1,0].imshow(F_y,extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[1,0].set_title(r'$\bf{C}$)  $F_y$')

axs[1,1].imshow(F_xy,extent=[xx[0],xx[-1],xx[0],xx[-1]],origin='lower',cmap='gray',vmin=-5,vmax=5)
axs[1,1].set_title(r'$\bf{D}$)  $F_{xy}$')

for axi in axs.flatten(): axi.set(xlabel='x',ylabel='y'), axi.axis('auto')

plt.tight_layout()
plt.savefig('multiint_visualizationEx1scipyNorm.png')
plt.show()

# Figure 19.7: Second example of multivariable indefinite integrals

In [ ]:
# x and y grids and dx
xx = np.linspace(-2*np.pi,2*np.pi,81)
dx = xx[1] - xx[0]

yy = np.linspace(-4,1.5,151)
dy = yy[1] - yy[0]

# the function
X,Y = np.meshgrid(xx,yy)
f_xy = 3*Y*X**2 + 5*np.sin(X*Y) + 2*Y*np.exp(-Y**2) + 4

# reminder that first dimension is for Y and second dimension is for X

# partial integrals and normalize
F_x = spi.cumulative_simpson(f_xy,dx=dx,axis=1,initial=True)
F_x -= F_x[:,np.argmin(abs(xx))].reshape(-1,1)

F_y = spi.cumulative_simpson(f_xy,dx=dy,axis=0,initial=True)
F_y -= F_y[np.argmin(abs(yy)),:].reshape(1,-1)

F_xy = spi.cumulative_simpson(F_x,dx=dx,axis=0,initial=True)
F_xy -= F_xy[np.argmin(abs(yy)),:].reshape(1,-1)
F_xy -= F_xy[:,np.argmin(abs(xx))].reshape(-1,1)

In [ ]:
# and now visualize
_,axs = plt.subplots(2,2,figsize=(10,6))

clim = 10

axs[0,0].imshow(f_xy,extent=[xx[0],xx[-1],yy[0],yy[-1]],origin='lower',cmap='gray',vmin=-clim,vmax=clim)
axs[0,0].set_title(r'$\bf{A}$)  $f(x,y)$')

axs[0,1].imshow(F_x,extent=[xx[0],xx[-1],yy[0],yy[-1]],origin='lower',cmap='gray',vmin=-clim,vmax=clim)
axs[0,1].set_title(r'$\bf{B}$)  $F_x $')

axs[1,0].imshow(F_y,extent=[xx[0],xx[-1],yy[0],yy[-1]],origin='lower',cmap='gray',vmin=-clim,vmax=clim)
axs[1,0].set_title(r'$\bf{C}$)  $F_y$')

axs[1,1].imshow(F_xy,extent=[xx[0],xx[-1],yy[0],yy[-1]],origin='lower',cmap='gray',vmin=-clim,vmax=clim)
axs[1,1].set_title(r'$\bf{D}$)  $F_{xy}$')

for axi in axs.flatten(): axi.set(xlabel='x',ylabel='y'), axi.axis('auto')

plt.tight_layout()
plt.savefig('multiint_visualizationEx2.png')
plt.show()

# Definite integrals

In [ ]:
# the function
x,y = sym.symbols('x,y')
f = 2*x + y**2

# integration bounds
x_a,x_b = -1,3
y_a,y_b = 0,2

sympy_int = sym.integrate(f,(x,x_a,x_b),(y,y_a,y_b))

# the result
display(Math('%s = %s' %(sym.latex(sym.Integral(f,(x,x_a,x_b),(y,y_a,y_b))),
                         sym.latex(sympy_int) ) ))

# Numerical approximation in numpy

In [ ]:
# recode in sympy
x,y = sym.symbols('x,y')
f = 2*x + y**2

# integration bounds
x_a,x_b = -1,3
y_a,y_b = 0,2

In [ ]:
# in numpy
xx = np.linspace(x_a,x_b,1601)
yy = np.linspace(y_a,y_b,1600)
X,Y = np.meshgrid(xx,yy)
fxy = sym.lambdify((x,y),f)(X,Y)

intOverX = np.sum(fxy,axis=1) * (xx[1]-xx[0])
doubleInt = np.sum(intOverX)  * (yy[1]-yy[0])

# or:
fullInt = np.sum(fxy) * (xx[1]-xx[0]) * (yy[1]-yy[0])

print(f'Sympy exact result: {sympy_int:.5f}')
print(f'Numpy in two steps: {doubleInt:.5f}')
print(f'Scipy in one step : {fullInt:.5f}')

# Figure 19.8: Definite integral in a region

In [ ]:
# function to draw the rectangle
import matplotlib.patches as patches

xx = np.linspace(x_a-2,x_b+3,1601)
yy = np.linspace(y_a-2,y_b+3,1600)

dx = xx[1]-xx[0]
dy = yy[1]-yy[0]

X,Y = np.meshgrid(xx,yy)
fxy = sym.lambdify((x,y),f)(X,Y)

# indices for integration bounds
x_idx = (xx >= x_a) & (xx <= x_b)
y_idx = (yy >= y_a) & (yy <= y_b)

# simple sum using numpy
npApprox = np.sum(fxy[np.ix_(y_idx,x_idx)]) * (dx*dy)

# double integrate in scipy on 1D at a time
F_x      = spi.simpson(fxy[:,x_idx], dx=dx, axis=1)
spApprox = spi.simpson(F_x[y_idx], dx=dy)

print(f'Sympy exact result : {sympy_int:.5f}')
print(f'Numpy approximation: {npApprox:.5f}')
print(f'Scipy approximation: {spApprox:.5f}')


# show the plot
fig,ax = plt.subplots(1,figsize=(5,4))
h = ax.imshow(fxy,origin='lower',extent=[xx[0],xx[-1],yy[0],yy[-1]],vmin=-5,vmax=5,cmap='gray')
ax.set(xlabel='x',ylabel='y')
ch = fig.colorbar(h,ax=ax,fraction=.0352)
ch.ax.tick_params(labelsize=10)

ax.add_patch( patches.Rectangle(
    (x_a,y_a),x_b-x_a,y_b-y_a,
    linestyle='--',linewidth=1,edgecolor='k',facecolor='none') )

plt.tight_layout()
plt.savefig('multiint_defIntScipy.png')
plt.show()

# Figure 19.9: Definite integral with variable bounds

In [ ]:
# Define variables and the function
x,y = sym.symbols('x,y')
fxy = x**2 + y

x_a = y**2
x_b = y + 0
y_a = 0
y_b = 1

# exact integral
inner_integral = sym.integrate(fxy,(x,x_a,x_b))
sym_defint = sym.integrate(inner_integral,(y,y_a,y_b))

# note: the above two lines can be condensed into one:
#sym_def = sym.integrate(fxy,(x,x_a,x_b),(y,y_a,y_b))

# Print the symbolic result
display(Math('%s ' %sym.latex(sym.Integral(sym.Integral(fxy,(x,x_a,x_b)),(y,y_a,y_b)))))
print('')
display(Math('%s = %s' %(sym.latex(sym.Integral(fxy,(x,x_a,x_b))),sym.latex(inner_integral))))
print('')
display(Math('%s = %s' %(sym.latex(sym.Integral(inner_integral,(y,y_a,y_b))),sym.latex(sym_defint))))

In [ ]:
# Define the region of integration
xx4region = np.linspace(y_a,y_b,100)
yLo4region = np.array([x_a.subs(y,yi) for yi in xx4region],dtype=float)
yHi4region = np.array([x_b.subs(y,yi) for yi in xx4region],dtype=float)

# Create a meshgrid for plotting the function
xx = np.linspace(float(x_a.subs(y,y_a)),float(x_b.subs(y,y_b)),499)
yy = np.linspace(y_a,y_b,499)
X,Y = np.meshgrid(xx,yy)
fxy_lam = sym.lambdify((x,y),fxy)
Z = fxy_lam(X,Y)


### visualization
fig,ax = plt.subplots(figsize=(12,6))

# function as heatmap
h = ax.imshow(Z,origin='lower',extent=[xx[0],xx[-1],yy[0],yy[-1]],
              cmap='gray',vmin=0,vmax=2,aspect='auto',alpha=.8)
fig.colorbar(h, ax=ax, label=r'$z = f(x,y)$')

# region of integration
ax.plot(xx4region,yLo4region,'k--',label=r'Lower bound = $%s$' %sym.latex(x_a))
ax.plot(xx4region,yHi4region,'k',label=r'Upper bound = $%s$' %sym.latex(x_b))
ax.fill_between(xx4region,yHi4region,yLo4region,color='k',alpha=.2,label='Integration window')
ax.legend()

# etc etc
ax.set(xlabel='$x$',ylabel='$y$',xlim=[0,1],ylim=[0,1])
ax.set_title(r'$\int_{%s}^{%s}\int_{%s}^{%s} \left(%s\right) \,dx\,dy = %s$'
             %(sym.latex(y_a),sym.latex(y_b),sym.latex(x_a),sym.latex(x_b),sym.latex(fxy),sym_defint),
             loc='center')

plt.tight_layout()
plt.savefig('multiint_variableIntegration1.png')
plt.show()

# Figure 19.11: Numerical approximation

In [ ]:
# Integration bounds
y_vals = np.linspace(y_a,y_b,1000)
slice_integrals = np.zeros(len(y_vals))

# iterate over y values, integrate along x
for idx,yi in enumerate(y_vals):

  # x bounds for this y
  lo_bnd = float(x_a.subs(y,yi))
  hi_bnd = float(x_b.subs(y,yi))
  x_vals = np.linspace(lo_bnd,hi_bnd,100)

  # function values for these coords
  f_vals = fxy_lam(x_vals,yi)

  # get the "mini-integral" of this slice
  slice_integrals[idx] = spi.simpson(f_vals,x=x_vals)

# Integrate the results along y
integral_result = spi.simpson(slice_integrals,x=y_vals)

# print the results
print(f'Sympy exact result : {sym_defint:.10f}')
print(f'Scipy approximation: {integral_result:.10f}')

In [ ]:
# note: this figure was made setting x_vals N=1000
plt.figure(figsize=(5,5))
plt.plot(slice_integrals[::10],y_vals[::10],'ks-',markerfacecolor='w',alpha=.6)
plt.gca().set(ylim=[y_a-.02,y_b+.02],ylabel=r'$y$-axis coordinate of horizontal slice',xlabel=r'Definite integral across $x$')

plt.tight_layout()
plt.savefig('multiint_variableIntSliceInts.png')
plt.show()

# Figure 19.12: The mask

In [ ]:
# create and visualize the mask
mask = (Y>=X**2) & (Y<=X)
plt.figure(figsize=(5,5))
plt.imshow(mask,origin='lower',extent=[xx[0],xx[-1],yy[0],yy[-1]],cmap='gray')
plt.tight_layout()
plt.savefig('multiint_variableIntMask.png')

# numpy approximation
dx = xx[1]-xx[0]
dy = yy[1]-yy[0]
integral_np = np.sum(Z[mask]) * dx*dy

# print the results
print(f'Sympy exact result : {sym_defint:.10f}')
print(f'Scipy approximation: {integral_result:.10f}')
print(f'Numpy approximation: {integral_np:.10f}')